# [Module 3] Prophet 과 DeepAP+ Predictor 생성 (Create predcitors of Prophet and DeepAR+)
- 여기서는 두개의 Predictos 인 Prophet 과 DeepAR+를 생성 (포케스팅 모델 학습) 을 하고, 실제 Predictor의 성능 지표를 통해서 얼마나 성능이 나왔는지를 학인 할 수 있습니다.
* 이 과정은 약 40분 정도 걸립니다. **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables 
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Hyperparameters
- forecastHorizon 는 31 개의 미래 데이타 포인트를 예측 합니다. ForecastFrequency 가 Day 기준이므로 미래의 31일을 예측 합니다.
- NumberOfBacktestWindows 는 백테스트를 의미합니다. 학습 데이타의 맨 마지막에서 2번 백테스트 합니다.
![BackTest](img/Fig3.1.BackTest.png)

In [4]:
forecastHorizon = 31 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 31 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "D"

Detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)

#### Predictor algorithms
- 아래 두개의 알고리즘을 사용 합니다.

In [5]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet
- Prophet을 학습하기 위해서 필요한 설정 파일들을 기술 합니다. 

In [6]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + suffix

In [8]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
        
      InputDataConfig= {"DatasetGroupArn": datasetGroupArn, 
                        #"SupplementaryFeatures": [ 
                        # { 
                        #    "Name": "holiday",
                        #    "Value": "US"
                        # } #]
                       },
      
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )




### DeepAR+

In [27]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix


In [52]:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      
      InputDataConfig= {"DatasetGroupArn": datasetGroupArn, 
                        
                        #"SupplementaryFeatures": [ 
                        #{ 
                        #  "Name": "holiday",
                        #    "Value": "US"
                        #} ]
                       },
     
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              },
                              {"AttributeName": "is_holidays", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"futurefill": "zero",                                     
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreatePredictor operation: A predictor already exists with the arn: arn:aws:forecast:ap-northeast-2:870180618679:predictor/StoreItemDemand_deeparp_algo_188082

In [20]:
### Optional ###
# DeepAR+ Specifics
#deeparp_predictorName= project+'_deeparp_algo_2' + suffix

In [22]:
### Optional ###
# Build DeepAR+:
'''
deeparp_create_predictor_response_2=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=True,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      
      InputDataConfig= {"DatasetGroupArn": datasetGroupArn, 
                        
                        #"SupplementaryFeatures": [ 
                        #{ 
                        #  "Name": "holiday",
                        #    "Value": "US"
                        #} ]
                       },
      TrainingParameters={"likelihood": "negative-binomial",
                            "num_averaged_models": "5"},
      HPOConfig={ "ParameterRanges": {
                        "ContinuousParameterRanges": [{
                            "Name": "learning_rate",
                            "MaxValue": 0.01,
                            "MinValue": 0.0001,
                            "ScalingType": "Auto"}],
                        "IntegerParameterRanges": [{
                            "Name": "context_length",
                            "MaxValue": 52,
                            "MinValue": 8,
                            "ScalingType": "Auto"}]
                        }
                    },
     
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                 }
                                ]
                              }
                            ]
                           }
                                                 )


'''

## Check if the predictors are active
- Predictor가 실제로 생성되고 있는 상태를 확인 합니다.

In [ ]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [17]:
prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]

In [18]:
deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [23]:
### Optional ###
#deepar_predictorArn_2=deeparp_create_predictor_response_2["PredictorArn"]

In [46]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= deepar_predictorArn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(120)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(120)
    else:
        break

Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 201 ms, sys: 18.7 ms, total: 220 ms
Wall time: 24min 2s


## Predictor evaluation
-아래 화면은 Forecast Console --> Dataset Group 선택 --> Predictors 선택을 하시면 조금 더 쉽게 볼 수 있습니다. 
![PredictorMetric](img/Fig.3.2.PredictorMetric.png)

#### Prophet

In [47]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 8.472997207678123,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05085495942140343,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.10930294131422662,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.04675069824538183,
                                                                                          'Quantile': 0.1}]}},
                                            

#### DeepAR+

In [48]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 9.344742353489487,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05282173557607016,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.12058968484880864,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.055090369807479854,
                                                                                          'Quantile': 0.1}]}},
                                      

In [49]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics_2 = forecast.get_accuracy_metrics(PredictorArn=deepar_predictorArn_2)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics_2)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 9.673564284242527,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.060579921063119276,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.12373088930613638,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.050260567676424436,
                                                                                          'Quantile': 0.1}]}},
                                     

In [51]:
%store prophet_predictorArn
%store deepar_predictorArn
#%store deepar_predictorArn_2


Stored 'prophet_predictorArn' (str)
Stored 'deepar_predictorArn' (str)
